In [157]:
import numpy as np
import pandas as pd

# Game Deck

In [158]:
class GameDeck():
    """ Create the game deck """
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values, suits=4):
        self.num_decks = num_decks
        self.cards = values * num_decks * suits
        
    def __str__(self):
        return "{} cards left\n{}".format(len(self.cards), self.cards)
    
    def shuffle_cards(self):
        np.random.shuffle(self.cards)
        
    def deal_card(self, hand=None):
        if hand == None:
            return self.cards.pop(0)
        else:
            hand.append(self.cards.pop(0))

In [159]:
test = GameDeck()

In [160]:
print (test)
test.shuffle_cards()
print ()
print (test)
test.deal_card()
print()
print (test)

52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']

52 cards left
[3, 8, 8, 7, 'Q', 8, 'K', 6, 'J', 10, 2, 4, 8, 'J', 'J', 'K', 'K', 4, 5, 10, 'Q', 'K', 'A', 6, 6, 9, 4, 'J', 2, 5, 7, 'Q', 7, 10, 'A', 4, 3, 6, 2, 5, 'Q', 2, 7, 3, 'A', 5, 'A', 3, 10, 9, 9, 9]

51 cards left
[8, 8, 7, 'Q', 8, 'K', 6, 'J', 10, 2, 4, 8, 'J', 'J', 'K', 'K', 4, 5, 10, 'Q', 'K', 'A', 6, 6, 9, 4, 'J', 2, 5, 7, 'Q', 7, 10, 'A', 4, 3, 6, 2, 5, 'Q', 2, 7, 3, 'A', 5, 'A', 3, 10, 9, 9, 9]


# Points calculator

In [446]:
def s_to_l(hand):
    try:
        hand = hand.split(",")
    except:
        pass
    
    working_hand = []
    
    for i in hand:
        try:
            working_hand.append(int(i))
        except:
            working_hand.append(i)
    return working_hand

In [447]:
def calc_ace(hand, count_A):
    # For each Ace, see if adding 11 causes it to bust
    # If no, then take Ace as 11
    # Else, take Ace as 1
    working_hand = hand.copy()
    
    # Remove Aces
    working_hand = [i if type(i) == int else 10 for i in working_hand if i != "A"]
    for k in range(count_A):
        if sum(working_hand) + 11 <= 21:
            working_hand.append(11)
        else:
            working_hand.append(1)
    return working_hand

In [448]:
def calc_points(hand):
    working_hand = s_to_l(hand)
    
    if "A" in working_hand:
        
        # Count number of Aces
        count_A = working_hand.count("A")
        working_hand = calc_ace(working_hand, count_A)
        
    else:
        # Replace J, Q, and K with 10
        working_hand = [10 if i in ["J", "Q", "K"] else int(i) for i in working_hand]
    
    return sum(working_hand)

In [449]:
for i in [["A", "K"], [10,10], ["A", 5], ["A", "A", 8], [3,3]]:
    print (i, calc_points(i))

['A', 'K'] 21
[10, 10] 20
['A', 5] 16
['A', 'A', 8] 20
[3, 3] 6


# Soft 17

- Some casinos require the dealer to hit on a soft 17
- We need to know when it is a soft 17

In [450]:
def soft_17(hand):
    working_hand = s_to_l(hand)
    
    Ace_11 = False
    if calc_points(working_hand) == 17 and "A" in working_hand:
        count_A = working_hand.count("A")
        working_hand = calc_ace(working_hand, count_A)
        if 11 in working_hand[-count_A:]:
            return True
    return False

In [451]:
for i in [["K", 7], ["A", 3, 3], ["A", "A", 5], ["A", "A", "A", 4]]:
    print (i, soft_17(i))

['K', 7] False
['A', 3, 3] True
['A', 'A', 5] True
['A', 'A', 'A', 4] True


# Play game

In [452]:
def start_game(num_decks=1):
    game_deck = GameDeck(num_decks=num_decks)
    
    d_hand = []
    p_hand = []
    
    game_deck.shuffle_cards()
    
    for _ in range(2):
        game_deck.deal_card(p_hand)
        game_deck.deal_card(d_hand)
        
    return game_deck, p_hand, d_hand

In [453]:
start_game()

(<__main__.GameDeck at 0x1111fb438>, ['Q', 'J'], [8, 4])

- strategies
    - 0: random
    - 1: recommended
    - 2: statistical analysis
    - 3: machine learning

In [454]:
def player_turn(game_deck, d_open, p_hand, strategy=0):
    new_hand = p_hand.copy()
    while calc_points(new_hand) <= 11:
        game_deck.deal_card(new_hand)
        
    if strategy == 0:
        while calc_points(new_hand) <= 18:
            if np.random.random() <= 0.5:
                game_deck.deal_card(new_hand)
            else:
                break
    elif strategy == 1:
        while calc_points(new_hand) <= 18:
            if d_open <= 6:
                game_deck.deal_card(new_hand)
            else:
                break
                
    return game_deck, p_hand, new_hand

In [455]:
test = start_game()
p_test = player_turn(test[0], test[1][0], test[2])
print (p_test)
print (calc_points(p_test[1]))
print (calc_points(p_test[2]))

(<__main__.GameDeck object at 0x110dd17b8>, [7, 9], [7, 9, 'J'])
16
26


In [456]:
def dealer_turn(game_deck, hand, soft=1):
    new_hand = hand.copy()
    if soft == 1:
        while calc_points(new_hand) < 17 or soft_17(new_hand):
            game_deck.deal_card(new_hand)
    else:
        while calc_points(new_hand) < 17:
            game_deck.deal_card(new_hand)
    return game_deck, hand, new_hand

In [457]:
test = start_game()
p_test = player_turn(test[0], test[1][0], test[2])
print (p_test)
print (calc_points(p_test[1]))
print (calc_points(p_test[2]))
d_test = dealer_turn(p_test[0], test[1])
print (d_test)
print (calc_points(d_test[1]))
print (calc_points(d_test[2]))

(<__main__.GameDeck object at 0x110a990b8>, [4, 8], [4, 8, 8])
12
20
(<__main__.GameDeck object at 0x110a990b8>, ['K', 7], ['K', 7])
17
17


In [458]:
def l_to_s(l):
    l = [str(i) for i in l]
    return ",".join(l)

In [459]:
def play_split(game_deck, p_hand, d_hand, d_open, num_decks, strategy=0, soft=1):
    to_split = np.random.random() <= 0.5
    if (strategy == 0 and to_split) or (strategy == 1 and p_hand[0] == 8):
        # Splits
        split = 1
        p_hand_1 = [p_hand[0]]
        p_hand_2 = [p_hand[1]]
        game_deck.deal_card(p_hand_1)
        game_deck.deal_card(p_hand_2)
        game_deck, p_hand_1, p_new_hand_1 = player_turn(game_deck, d_open, p_hand_1, strategy=strategy)
        game_deck, p_hand_2, p_new_hand_2 = player_turn(game_deck, d_open, p_hand_2, strategy=strategy)

        # If either is below 21, dealer gets to play
        if calc_points(p_new_hand_1) <= 21 or calc_points(p_new_hand_2) <= 21:
            game_deck, d_hand, d_new_hand = dealer_turn(game_deck, d_hand)
            # If hand 1 is above 21
            if calc_points(p_new_hand_1) > 21:
                # Dealer doesn't get to play for this hand
                d_new_hand_1 = d_hand.copy()
                out_1 = np.array([l_to_s(d_hand), l_to_s(d_new_hand_1), l_to_s(p_hand_1), l_to_s(p_new_hand_1), strategy, split, num_decks, soft])       
            else:
                out_1 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_1), l_to_s(p_new_hand_1), strategy, split, num_decks, soft])       
            # If hand 2 is above 21
            if calc_points(p_new_hand_2) > 21:
                # Dealer doesn't get to play for this hand
                d_new_hand_2 = d_hand.copy()
                out_2 = np.array([l_to_s(d_hand), l_to_s(d_new_hand_2), l_to_s(p_hand_2), l_to_s(p_new_hand_2), strategy, split, num_decks, soft])
            else:
                out_2 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_2), l_to_s(p_new_hand_2), strategy, split, num_decks, soft])       
        # They are both above 21, dealer doesn't play at all
        else:
            d_new_hand = d_hand.copy()
            out_2 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_2), l_to_s(p_new_hand_2), strategy, split, num_decks, soft])       
            out_1 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_1), l_to_s(p_new_hand_1), strategy, split, num_decks, soft])       
            
        
        return np.array([out_1, out_2])
    else:
        return "No split"

In [460]:
def card_value(card):
    try:
        return int(card)
    except:
        if card == "A":
            return 1
        else:
            return 10

In [461]:
def play_game(num_decks=4, strategy=0, soft=1):
    game_deck, p_hand, d_hand = start_game(num_decks=num_decks)
    
    d_open = card_value(d_hand[0])
    split = 0
    if calc_points(p_hand) != 21 and calc_points(d_hand) != 21:
        if (p_hand[0] == p_hand[1]):
            out = play_split(game_deck, p_hand, d_hand, d_open, num_decks, strategy=strategy, soft=soft)
        else:
            out = "No split"
        if type(out) == str:
            game_deck, p_hand, p_new_hand = player_turn(game_deck, d_open, p_hand, strategy=strategy)
            if calc_points(p_new_hand) <= 21:
                game_deck, d_hand, d_new_hand = dealer_turn(game_deck, d_hand, soft=soft)
            else:
                d_new_hand = d_hand.copy()
            out = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand), l_to_s(p_new_hand), strategy, split, num_decks, soft])
    else:
        p_new_hand = p_hand.copy()
        d_new_hand = d_hand.copy()
        out = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand), l_to_s(p_new_hand), strategy, split, num_decks, soft])
    
    return out

In [462]:
play_game(strategy=1)

array(['8,Q', '8,Q', '6,4', '6,4,Q', '1', '0', '4', '1'], 
      dtype='<U5')

In [463]:
a = play_game()
while a.ndim < 2:
    a = play_game()
print (a)

[['4,2' '4,2,Q,3' 'J,Q' 'J,Q' '0' '1' '4' '1']
 ['4,2' '4,2,Q,3' 'J,K' 'J,K' '0' '1' '4' '1']]


In [464]:
a.ndim

2

# Generate cards

In [465]:
def gen_df(num_games=5000):
    data = []
    for _ in range(num_games):
        a = play_game(strategy=np.random.randint(2), soft=np.random.randint(2))
        if a.ndim == 2:
            data.extend(a)
        else:
            data.append(a)
            
    df = pd.DataFrame(data, columns=["d_hand", "d_final_hand", "p_hand", "p_final_hand", "strategy", "split", "num_decks", "soft_17"])
    return df

In [466]:
gen_df(num_games=1)

,d_hand,d_final_hand,p_hand,p_final_hand,strategy,split,num_decks,soft_17
0,"2,Q","2,Q,7","K,5","K,5,6",1,0,4,1


# Add columns

In [467]:
def add_columns(original):
    df = original.copy()
    df["strategy"] = df["strategy"].astype(int)
    df["split"] = df["split"].astype(int)
    df["num_decks"] = df["num_decks"].astype(int)
    df["soft_17"] = df["soft_17"].astype(int)
    
    df["d_open"] = df["d_hand"].apply(lambda x: card_value(x.split(",")[0]))
    df["d_initial"] = df["d_hand"].apply(calc_points)
    df["d_final"] = df["d_final_hand"].apply(calc_points)
    df["d_hit"] = df["d_final"]-df["d_initial"] > 0
    df["d_hit"] = df["d_hit"].astype(int)
    df["d_bust"] = df["d_final"] > 21
    df["d_bust"] = df["d_bust"].astype(int)
    
    df["p_initial"] = df["p_hand"].apply(calc_points)
    df["p_final"] = df["p_final_hand"].apply(calc_points)
    df["p_hit"] = df["p_final"]-df["p_initial"] > 0
    df["p_hit"] = df["p_hit"].astype(int)
    df["p_bust"] = df["p_final"] > 21
    df["p_bust"] = df["p_bust"].astype(int)
    
    df["p_win"] = (df["p_final"] > df["d_final"]) & (df["p_bust"] == 0) | (df["d_bust"] == 1)
    df["p_win"] = df["p_win"].astype(int)

    return df

# Generate CSVs

In [551]:
df = gen_df()

In [552]:
try:
    old_df = pd.read_csv("cards.csv")
    df = pd.concat([df, old_df])
except:
    pass
df.to_csv("cards.csv", index=False)

In [557]:
df = pd.read_csv("cards.csv")

In [558]:
df.shape

(50994, 8)

In [559]:
df2 = add_columns(df)

In [560]:
df2.to_csv("blackjack_data.csv", index=False)

# Test data

In [510]:
df2.describe()

,strategy,split,num_decks,soft_17,d_open,d_initial,d_final,d_hit,d_bust,p_initial,p_final,p_hit,p_bust,p_win
count,5099.000000,5099.000000,5099.0,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000
mean,0.485389,0.038831,4.0,0.508923,6.600706,14.633458,18.556972,0.451265,0.202589,14.612865,18.571092,0.445185,0.200628,0.380467
std,0.499835,0.193211,0.0,0.499969,3.156163,4.088947,4.247913,0.497668,0.401968,4.042773,4.141357,0.497035,0.400509,0.485549
min,0.000000,0.000000,4.0,0.000000,1.000000,4.000000,4.000000,0.000000,0.000000,4.000000,4.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,4.0,0.000000,4.000000,12.000000,17.000000,0.000000,0.000000,12.000000,15.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,4.0,1.000000,7.000000,15.000000,19.000000,0.000000,0.000000,15.000000,19.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,4.0,1.000000,10.000000,18.000000,21.000000,1.000000,0.000000,18.000000,21.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,4.0,1.000000,10.000000,21.000000,26.000000,1.000000,1.000000,21.000000,28.000000,1.000000,1.000000,1.000000


## Check hands and points

In [470]:
print (df2[["d_hand", "d_initial"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['3,J' 13]
 ['K,8' 18]
 ['8,3' 11]
 ['7,7' 14]
 ['5,10' 15]]


In [471]:
print (df2[["d_final_hand", "d_final"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['7,K' 17]
 ['K,A' 21]
 ['4,J,10' 24]
 ['K,8' 18]
 ['6,Q,9' 25]]


In [472]:
print (df2[["p_hand", "p_initial"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['3,7' 10]
 ['3,6' 9]
 ['10,2' 12]
 ['9,7' 16]
 ['A,7' 18]]


In [473]:
print (df2[["p_final_hand", "p_final"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['2,5,K' 17]
 ['J,7,7' 24]
 ['2,3,J' 15]
 ['3,10' 13]
 ['J,7' 17]]


## If p_bust = 1, p_win = 0

In [474]:
df2.p_win[df2["p_bust"] == 1].value_counts()

0    1023
Name: p_win, dtype: int64

## If d_bust = 1, p_win = 1

In [475]:
df2.p_win[df2["d_bust"] == 1].value_counts()

1    1033
Name: p_win, dtype: int64

## If soft_17 = 1, then when d_final == 17, it's not a soft 17

- If dealer is required to hit on soft 17
    - If player didn't get blackjack
    - If player didn't bust
    - Dealer should not have a soft 17 final hand

In [488]:
for i in df2.d_final_hand[(df2.soft_17 == 1) & (df2.d_final == 17)].index:
    if soft_17(df2.ix[i, "d_final_hand"]) and df2.ix[i,"p_bust"] == 0 and df2.ix[i,"p_initial"] != 21:
        print (df2.ix[i])

## If no one busted, the higher point wins

In [499]:
sub = df2[(df2.p_bust == 0) & (df2.d_bust == 0) & (df2.p_win == 1)]

In [505]:
(sub.p_final > sub.d_final).value_counts()

True    907
dtype: int64

In [507]:
sub = df2[(df2.p_bust == 0) & (df2.d_bust == 0) & (df2.p_win == 0)]

In [509]:
(sub.p_final <= sub.d_final).value_counts()

True    2136
dtype: int64